In [1]:
from collections import dataclass
from pathlib import Path

ImportError: cannot import name 'dataclass' from 'collections' (c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\collections\__init__.py)

In [ ]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    raw_data:Path
    unzip_dir:Path


In [2]:
from insurence_premium.constant import *

In [ ]:
class ConfigurationManager:
    def __init__(
        self ,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
    )